In [47]:
import hashlib
import os
import tarfile
import zipfile
import requests

#@save
DATA_HUB = dict()
DATA_URL = 'http://d2l-data.s3-accelerate.amazonaws.com/'

In [48]:
def download(name, cache_dir=os.path.join('..', 'data')):  #@save
    """下载一个DATA_HUB中的文件，返回本地文件名"""
    assert name in DATA_HUB, f"{name} 不存在于 {DATA_HUB}"
    url, sha1_hash = DATA_HUB[name]
    os.makedirs(cache_dir, exist_ok=True)
    fname = os.path.join(cache_dir, url.split('/')[-1])
    if os.path.exists(fname):
        sha1 = hashlib.sha1()
        with open(fname, 'rb') as f:
            while True:
                data = f.read(1048576)
                if not data:
                    break
                sha1.update(data)
        if sha1.hexdigest() == sha1_hash:
            return fname  # 命中缓存
    print(f'正在从{url}下载{fname}...')
    r = requests.get(url, stream=True, verify=True)
    with open(fname, 'wb') as f:
        f.write(r.content)
    return fname

In [49]:
def download_extract(name, folder=None):  #@save
    """下载并解压zip/tar文件"""
    fname = download(name)
    base_dir = os.path.dirname(fname)
    data_dir, ext = os.path.splitext(fname)
    if ext == '.zip':
        fp = zipfile.ZipFile(fname, 'r')
    elif ext in ('.tar', '.gz'):
        fp = tarfile.open(fname, 'r')
    else:
        assert False, '只有zip/tar文件可以被解压缩'
    fp.extractall(base_dir)
    return os.path.join(base_dir, folder) if folder else data_dir

def download_all():  #@save
    """下载DATA_HUB中的所有文件"""
    for name in DATA_HUB:
        download(name)

In [50]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l

In [51]:
DATA_HUB['kaggle_house_train'] = (  #@save
    DATA_URL + 'kaggle_house_pred_train.csv',
    '585e9cc93e70b39160e7921475f9bcd7d31219ce')

DATA_HUB['kaggle_house_test'] = (  #@save
    DATA_URL + 'kaggle_house_pred_test.csv',
    'fa19780a7b011d9b009e8bff8e99922a8ee2eb90')

In [52]:
train_data = pd.read_csv(download('kaggle_house_train'))
test_data = pd.read_csv(download('kaggle_house_test'))

In [53]:
print(train_data.shape)
print(test_data.shape)

(1460, 81)
(1459, 80)


In [54]:
print(train_data.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]])

   Id  MSSubClass MSZoning  LotFrontage SaleType SaleCondition  SalePrice
0   1          60       RL         65.0       WD        Normal     208500
1   2          20       RL         80.0       WD        Normal     181500
2   3          60       RL         68.0       WD        Normal     223500
3   4          70       RL         60.0       WD       Abnorml     140000


In [55]:
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))

In [56]:
# 若无法获得测试数据，则可根据训练数据计算均值和标准差
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 在标准化数据之后，所有均值消失，因此我们可以将缺失值设置为0
all_features[numeric_features] = all_features[numeric_features].fillna(0)

In [57]:
# “Dummy_na=True”将“na”（缺失值）视为有效的特征值，并为其创建指示符特征
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape

(2919, 331)

In [58]:
# device = torch.device('cpu')

device = torch.device('mps')

In [59]:
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float32)
test_features = torch.tensor(all_features[n_train:].values, dtype=torch.float32)
train_labels = torch.tensor(
    train_data.SalePrice.values.reshape(-1, 1), dtype=torch.float32)

In [84]:
loss = nn.MSELoss()
in_features = train_features.shape[1]

def get_net():
    net = nn.Sequential(nn.Linear(in_features,1))
    return net

In [61]:
# 测试net的设备
# net.to(device=try_gpu())
net111 = get_net()
print(net111)
print(next(net111.parameters()).device)

Sequential(
  (0): Linear(in_features=331, out_features=1, bias=True)
)
mps:0


In [77]:
def log_rmse(net, features, labels):
    # 为了在取对数时进一步稳定该值，将小于1的值设置为1
    features, labels = features.to(device), labels.to(device)
    clipped_preds = torch.clamp(net(features), 1, float('inf'))
    rmse = torch.sqrt(loss(torch.log(clipped_preds),
                           torch.log(labels)))
    #print(rmse.device)
    return rmse.item()

In [86]:
def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    train_iter = d2l.load_array((train_features, train_labels), batch_size)
    # 这里使用的是Adam优化算法
    optimizer = torch.optim.Adam(net.parameters(),
                                 lr = learning_rate,
                                 weight_decay = weight_decay)
    net.train()
    for epoch in range(num_epochs):
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            l = loss(net(X), y)
            l.backward()
            optimizer.step()
        #print('net device', next(net.parameters()).device)
#         train_ls.append(log_rmse(net, train_features, train_labels))
#         if test_labels is not None:
#             test_ls.append(log_rmse(net, test_features, test_labels))
    return train_ls, test_ls

In [64]:
def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat([X_train, X_part], 0)
            y_train = torch.cat([y_train, y_part], 0)
    return X_train, y_train, X_valid, y_valid

In [85]:
def k_fold(k, X_train, y_train, num_epochs, learning_rate, weight_decay,
           batch_size):
    train_l_sum, valid_l_sum = 0, 0
    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train)
        net = get_net()
        net = net.to(device=device)
        train_ls, valid_ls = train(net, *data, num_epochs, learning_rate,
                                   weight_decay, batch_size)
        
        if len(train_ls) > 1:
            train_l_sum += train_ls[-1]
            valid_l_sum += valid_ls[-1]
#         if i == 0:
#             d2l.plot(list(range(1, num_epochs + 1)), [train_ls, valid_ls],
#                      xlabel='epoch', ylabel='rmse', xlim=[1, num_epochs],
#                      legend=['train', 'valid'], yscale='log')
#         print(f'折{i + 1}，训练log rmse{float(train_ls[-1]):f}, '
#               f'验证log rmse{float(valid_ls[-1]):f}')
    return train_l_sum / k, valid_l_sum / k

In [87]:
# CPU训练
device = torch.device('cpu')

timer = d2l.Timer()

k, num_epochs, lr, weight_decay, batch_size = 5, 10, 5, 0, 64
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr,
                          weight_decay, batch_size)
print(f'{k}-折验证: 平均训练log rmse: {float(train_l):f}, '
      f'平均验证log rmse: {float(valid_l):f}')

f'{timer.stop():.2f} sec'

5-折验证: 平均训练log rmse: 0.000000, 平均验证log rmse: 0.000000


'0.29 sec'

In [88]:
# GPU训练
device = torch.device('mps')

timer = d2l.Timer()

k, num_epochs, lr, weight_decay, batch_size = 5, 10, 5, 0, 64
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr,
                          weight_decay, batch_size)


print(f'{k}-折验证: 平均训练log rmse: {float(train_l):f}, '
      f'平均验证log rmse: {float(valid_l):f}')

f'{timer.stop():.2f} sec'

5-折验证: 平均训练log rmse: 0.000000, 平均验证log rmse: 0.000000


'1.70 sec'

In [46]:
# cpu训练 看看时间
timer = d2l.Timer()

hd_1, hd_2 = 256, 64
device_gpu = torch.device('cpu')
k, num_epochs, lr, weight_decay, batch_size = 5, 100, 5, 0, 64

n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float32, device=device_gpu)
test_features = torch.tensor(all_features[n_train:].values, dtype=torch.float32, device=device_gpu)
train_labels = torch.tensor(
    train_data.SalePrice.values.reshape(-1, 1), dtype=torch.float32, device=device_gpu)

def get_net():
    net = nn.Sequential(
        nn.Linear(in_features, hd_1), nn.ReLU(),
        nn.Linear(hd_1, hd_2), nn.ReLU(),
        nn.Linear(hd_2, 1),
    )
    return net.to(device=device_gpu)

train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr,
                          weight_decay, batch_size)
print(f'{k}-折验证: 平均训练log rmse: {float(train_l):f}, '
      f'平均验证log rmse: {float(valid_l):f}')

print(f'{timer.stop():.5f} sec')

5-折验证: 平均训练log rmse: 1.911470, 平均验证log rmse: 1.934578
8.67312 sec


In [41]:
num_epochs = 10
lr_s = [5.3, 5.2, 5.1, 5, 4.9, 4.8]
weight_decay_s = [0, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]

hd_1, hd_2 = 256, 64

def get_net():
    net = nn.Sequential(
        nn.Linear(in_features, hd_1), nn.ReLU(),
        nn.Linear(hd_1, hd_2), nn.ReLU(),
        nn.Linear(hd_2, 1),
    )
    return net.to(device=device_gpu)

for lr in lr_s:
    #print('lr is', lr)
    for weight_decay in weight_decay_s:
        #print('weight_decay is', weight_decay)
        train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr,
                                  weight_decay, batch_size)
        print(f'lr 是 {lr}, weight_decay 是 {weight_decay}, {k}-折验证: 平均训练log rmse: {float(train_l):f}, '
              f'平均验证log rmse: {float(valid_l):f}')
        print()

net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps

net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps:0
net device mps

KeyboardInterrupt: 